In [25]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

# Load dataset
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

# Inputs = all columns except first (ID) and last (target)
unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1]

# Balance the dataset
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
        indices_to_remove.append(i)

unscaled_inputs = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
unscaled_targets = np.delete(targets_all, indices_to_remove, axis=0)

# Scale inputs only
scaled_inputs = preprocessing.scale(unscaled_inputs)
scaled_targets = unscaled_targets  # keep as 0/1

# Shuffle
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = scaled_targets[shuffled_indices]

# Train/Val/Test split
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# Save
np.savez('Audiobooks_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_test', inputs=test_inputs, targets=test_targets)

# Load properly
npz = np.load('Audiobooks_train.npz')
train_inputs1, train_targets1 = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_validation.npz')
validation_inputs1, validation_targets1 = npz['inputs'].astype(float), npz['targets'].astype(int)

npz = np.load('Audiobooks_test.npz')
test_inputs1, test_targets1 = npz['inputs'].astype(float), npz['targets'].astype(int)

# Model
input_size = train_inputs1.shape[1]
output_size = 2
hidden_layer_size = 100
early_callback=tf.keras.callbacks.EarlyStopping(patience=3)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_inputs1, train_targets1, 
          validation_data=(validation_inputs1, validation_targets1),
          batch_size=100, epochs=50,callbacks=[early_callback],verbose=2)


Epoch 1/50
36/36 - 1s - loss: 0.4643 - accuracy: 0.8268 - val_loss: 0.2766 - val_accuracy: 0.9060 - 1s/epoch - 40ms/step
Epoch 2/50
36/36 - 0s - loss: 0.3142 - accuracy: 0.8846 - val_loss: 0.2583 - val_accuracy: 0.9150 - 200ms/epoch - 6ms/step
Epoch 3/50
36/36 - 0s - loss: 0.2783 - accuracy: 0.8949 - val_loss: 0.2511 - val_accuracy: 0.9105 - 213ms/epoch - 6ms/step
Epoch 4/50
36/36 - 0s - loss: 0.2699 - accuracy: 0.8969 - val_loss: 0.2361 - val_accuracy: 0.9128 - 269ms/epoch - 7ms/step
Epoch 5/50
36/36 - 0s - loss: 0.2544 - accuracy: 0.9039 - val_loss: 0.2517 - val_accuracy: 0.9105 - 276ms/epoch - 8ms/step
Epoch 6/50
36/36 - 0s - loss: 0.2503 - accuracy: 0.9064 - val_loss: 0.2539 - val_accuracy: 0.9150 - 256ms/epoch - 7ms/step
Epoch 7/50
36/36 - 0s - loss: 0.2507 - accuracy: 0.9070 - val_loss: 0.2380 - val_accuracy: 0.9195 - 290ms/epoch - 8ms/step


In [26]:
test_loss,test_accuracy=model.evaluate(test_inputs1,test_targets1)

14/14 [==============================] - 0s 4ms/step - loss: 0.2525 - accuracy: 0.8973
